# Import Dependencies

In [80]:
import pandas as pd
from sodapy import Socrata
import requests
import json
import datetime
import numpy as np
from urllib.request import Request, urlopen
from time import sleep
import credentials
pd.set_option("display.max_columns", None)

In [81]:
#list of zip codes in Montgomery County to be used for processing data to avoid 
moco_zips = ['20705','20707','20777','20783','20812','20814','20815','20816','20817','20818','20832','20833','20837','20838',
             '20839','20841','20842','20850','20851','20852','20853','20854','20855','20860','20861','20862','20866','20868',
             '20871','20872','20874','20876','20877','20878','20879','20882','20886','20895','20901','20902','20903','20904',
             '20905','20906','20910','20912','21771','21797','20813','20824','20825','20827','20830','20847','20848','20849',
             '20859','20875','20880','20883','20884','20885','20891','20896','20898','20907','20908','20911','20913','20914',
             '20915','20916','20918','20810','20811','20857','20889','20892','20894','20897','20899','20993','20997']

# Datasets

## Residential Permit Dataset

In [3]:
#authenticate client
client = Socrata('data.montgomerycountymd.gov',credentials.data_moco_token,
                 credentials.data_moco_username,credentials.data_moco_password)

#API Call for all residential permits
res_permits = client.get_all("m88u-pqki")

#convert response to dataframe
res_permits_df = pd.DataFrame.from_records(res_permits)

## Clean Residential Permit Dataset

In [82]:
#drop location columns
trimmed_res_permits_df = res_permits_df.drop('location',axis='columns', inplace = False)

#drop duplicate entries
no_dups_res_permits_df = trimmed_res_permits_df.drop_duplicates(inplace=False)

#create dataframe of new constrcutions excluding entries with missing zip codes and finaled date
res_construct_df = no_dups_res_permits_df.loc[(no_dups_res_permits_df['zip'].notnull()) & 
                                              (no_dups_res_permits_df['worktype'] == 'CONSTRUCT') &
                                             (no_dups_res_permits_df['finaleddate'].notnull())]

#create dataframe with the without incorrectly written zip codes
adjusted_res_construct_df = res_construct_df.loc[res_construct_df['zip'].isin(moco_zips)]

#convert finaled date column to datetime formate
adjusted_res_construct_df['finaleddate'] = pd.to_datetime(adjusted_res_construct_df['finaleddate'])

#contruction projects after 1/1/2016
filtered_res_construct_df = adjusted_res_construct_df.loc[
    adjusted_res_construct_df['finaleddate'] >= datetime.datetime(2016, 1, 1)]

#table with number of homes constructed by zip code
res_constuction_freq_series = filtered_res_construct_df.groupby('zip')['worktype'].value_counts()

#convert series to dataframe to merge
res_constuction_freq_df = res_constuction_freq_series.to_frame().rename(columns={'worktype':'count'}).reset_index()

#drop worktype column
res_constuction_freq_df.drop(['worktype'], axis = 1, inplace = True)

C:\Users\mivul\AppData\Local\Temp/ipykernel_10024/1865923425.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjusted_res_construct_df['finaleddate'] = pd.to_datetime(adjusted_res_construct_df['finaleddate'])


In [83]:
#display dataframe
res_constuction_freq_df.head()

,zip,count
0,20707,5
1,20812,12
2,20814,444
3,20815,326
4,20816,220


## Commercial Permit Dataset

In [84]:
#authenticate client
client = Socrata('data.montgomerycountymd.gov',credentials.data_moco_token,
                 credentials.data_moco_username,credentials.data_moco_password)

#Api Call for all commercial permits
comm_permits = client.get_all("i26v-w6bd")

#convert response to dataframe
comm_permits_df = pd.DataFrame.from_records(comm_permits)

#display dataframe
comm_permits_df.head()

,permitno,status,stno,stname,suffix,city,state,zip,addeddate,buildingarea,declaredvaluation,description,applicationtype,worktype,usecode,location,:@computed_region_a9cs_3ed7,:@computed_region_vu5j_pcmz,:@computed_region_tx5f_5em3,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n,issueddate,predir,finaleddate,postdir
0,902464,Open,12623,WISTERIA,DR,GERMANTOWN,MD,20874,2020-01-17T12:01:41.000,5950,185000,"Customer Wants To Use ePlans,fit out a premium...",COMMERCIAL BUILDING,ALTER,BUSINESS BUILDING,"{'latitude': '39.170095', 'longitude': '-77.26...",2,2,2,39,39,1,NaN,NaN,NaN,NaN
1,808314,Issued,2701,HUME,DR,SILVER SPRING,MD,20910,2017-07-28T02:07:02.000,13152,804470,"Revision ,3 - window revision per discussion w...",COMMERCIAL BUILDING,ALTER,MULTIFAMILY DWELLING LOW RISE,"{'latitude': '39.011375', 'longitude': '-77.05...",1,1,1,6,3065,1,2018-04-10T12:04:19.000,NaN,NaN,NaN
2,902537,Finaled,5411,CEDAR,LN,BETHESDA,MD,20814,2020-01-20T08:01:54.000,2300,500000,"Customer Wants To Use ePlans,Remove and replac...",COMMERCIAL BUILDING,RESTORE AND / OR REPAIR,BUSINESS BUILDING,"{'human_address': '{""address"": ""5411 CEDAR LN""...",NaN,NaN,NaN,NaN,NaN,NaN,2020-08-06T11:08:27.000,W,2022-04-05T12:04:36.000,NaN
3,902417,Issued,7272,WISCONSIN,AVE,BETHESDA,MD,20814,2020-01-17T11:01:25.000,19241,2205272,"Revision ,2 - Revision to an issued permit to ...",COMMERCIAL BUILDING,ALTER,BUSINESS BUILDING,"{'latitude': '38.982289', 'longitude': '-77.09...",1,1,1,5,111,1,2020-04-17T12:04:55.000,NaN,NaN,NaN
4,902492,Finaled,9605,MEDICAL CENTER,DR,ROCKVILLE,MD,20850,2020-01-17T03:01:22.000,20562,106000,Customer Wants To Use ePlans Interior work onl...,COMMERCIAL BUILDING,ALTER,BIOSCIENCE,"{'latitude': '39.10368', 'longitude': '-77.196...",3,3,3,25,71,1,2020-02-03T11:02:55.000,NaN,2020-04-06T02:04:23.000,NaN


## Clean Commercial Permit Dataset

In [85]:
#drop location columns
trimmed_comm_permits_df = comm_permits_df.drop('location',axis='columns', inplace = False)

#drop duplicate entries
no_dups_comm_permits_df = trimmed_comm_permits_df.drop_duplicates(inplace=False)

#create df of new constrcutions
com_construct_df = no_dups_comm_permits_df.loc[(no_dups_comm_permits_df['zip'].notnull()) & 
                                               (no_dups_comm_permits_df['worktype'] == 'CONSTRUCT')  &
                                                 (no_dups_comm_permits_df['finaleddate'].notnull())]

#dataframe with the incorrectly written zip codes removed
adjusted_com_construct_df = com_construct_df.loc[com_construct_df['zip'].isin(moco_zips)]

#convert finaled date column to datetime formate
adjusted_com_construct_df['finaleddate'] = pd.to_datetime(adjusted_com_construct_df['finaleddate'])

#contruction projects after 1/1/2016
filtered_com_construct_df = adjusted_com_construct_df.loc[
    adjusted_com_construct_df['finaleddate'] >= datetime.datetime(2016, 1, 1)]


#count new commerical constructions by zip code
com_constuction_freq_series = filtered_com_construct_df.groupby('zip')['worktype'].value_counts()

#convert series to dataframe to merge
com_constuction_freq_df = com_constuction_freq_series.to_frame().rename(columns={'worktype':'count'}).reset_index()

#drop worktype column
com_constuction_freq_df.drop(['worktype'], axis = 1, inplace = True)

In [87]:
#display dataframe
com_constuction_freq_df.head()

,zip,count
0,20814,100
1,20815,57
2,20816,9
3,20817,141
4,20818,1


## Merge Commercial and Residential construction

In [88]:
#Union merge residential and commercial construction dataframes 
complete_construction_df = com_constuction_freq_df.merge(res_constuction_freq_df,  how='outer', on='zip')

## Clean Merged table

In [89]:
#fill in missing values with zero
filled_complete_construction_df = complete_construction_df.fillna(0)

#create new columns with total residential and commercial construction by zip
filled_complete_construction_df['total'] = filled_complete_construction_df[['count_x','count_y']].agg('sum', axis = 1)

#drop columns
final_construction_df = filled_complete_construction_df.drop(['count_x','count_y'], axis = 1)

#Normalize building total
final_construction_df['total'] = final_construction_df['total'].apply(
    lambda x: (x-min(final_construction_df['total']))/(max(final_construction_df['total'])
                                                       -min(final_construction_df['total'])))

In [91]:
#display dataframe
final_construction_df.head()

,zip,total
0,20814,0.214909
1,20815,0.151071
2,20816,0.090008
3,20817,0.358049
4,20818,0.015067


## Food Inspection Dataset

In [27]:
#authenticate client
client = Socrata('data.montgomerycountymd.gov',credentials.data_moco_token,
                 credentials.data_moco_username,credentials.data_moco_password)

#API Call for all food inspections
food_inspection = client.get_all("5pue-gfbe")

#convert response to dataframe
food_df = pd.DataFrame.from_records(food_inspection)

#display dataframe
food_df.head()

,establishment_id,name,address1,city,zip,inspectiondate,inspectionresults,violation1,violation2,violation3,violation4,violation5,violation6a,violation6b,violation7a,violation7b,violation8,violation9,violation20,violation22,violationmenu,violationtransfat,violationsmoking,inspectiontype,organization,category,type,latitude,longitude,location,:@computed_region_vu5j_pcmz,:@computed_region_tx5f_5em3,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n,:@computed_region_d9ke_fpxt,address2
0,37032,168 ASIAN BURRITO,18000 GEORGIA AVE.,OLNEY,20832,2022-01-26T00:00:00.000,Critical Violations Corrected,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Out of Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Not applicable,In Compliance,In Compliance,Comprehensive,168 ASIAN BURRITO LLC,Market,Food,39.1511,-77.0673,"{'latitude': '39.1511', 'longitude': '-77.0673...",4,4,58,100,1,1,NaN
1,37032,168 ASIAN BURRITO,18000 GEORGIA AVE.,OLNEY,20832,2021-08-24T00:00:00.000,Critical Violations Corrected,In Compliance,Out of Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Not applicable,In Compliance,In Compliance,Monitoring,168 ASIAN BURRITO LLC,Market,Food,39.1511,-77.0673,"{'latitude': '39.1511', 'longitude': '-77.0673...",4,4,58,100,1,1,NaN
2,37032,168 ASIAN BURRITO,18000 GEORGIA AVE.,OLNEY,20832,2021-03-11T00:00:00.000,No Critical Violations Noted,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Not applicable,In Compliance,In Compliance,Monitoring,168 ASIAN BURRITO LLC,Market,Food,39.1511,-77.0673,"{'latitude': '39.1511', 'longitude': '-77.0673...",4,4,58,100,1,1,NaN
3,37032,168 ASIAN BURRITO,18000 GEORGIA AVE.,OLNEY,20832,2020-09-30T00:00:00.000,Critical Violations Corrected,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Out of Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Not applicable,In Compliance,In Compliance,Comprehensive,168 ASIAN BURRITO LLC,Market,Food,39.1511,-77.0673,"{'latitude': '39.1511', 'longitude': '-77.0673...",4,4,58,100,1,1,NaN
4,32992,29 CONVENIENCE MART,10755 COLESVILLE RD,SILVER SPRING,20901,2022-02-03T00:00:00.000,No Critical Violations Noted,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Not applicable,In Compliance,In Compliance,Monitoring,A&I ENTERPRISE VI INC - ALEENA MALIK,Market,Food,39.032,-77.0034,"{'latitude': '39.032', 'longitude': '-77.0034'...",5,5,18,21,1,1,NaN


## Clean Food Inspection Dataset

In [92]:
#drop undesired columns
trimmed_food_df = food_df.drop('location',axis='columns', inplace = False)

#replace a zip code value
trimmed_food_df.replace("209024510", "20902", inplace = True)

#drop duplicate stores to get a list of unique establishments
no_dups_food_df = trimmed_food_df.drop_duplicates(inplace=False)

#list for establishment categories of interest
store_categories = ['Restaurant','Market', 'Farmers Market', 'Carry Out']

#filter dataframe for desired location categories and create new dataframe
clean_food_df = no_dups_food_df.loc[no_dups_food_df.category.isin(store_categories) & no_dups_food_df.zip.isin(moco_zips)]

#number of food location category type by zipcode
final_food_df=clean_food_df.groupby('zip')["category"].value_counts().unstack().fillna(0)

#normalize food category numbers
for column in final_food_df.columns:
    if column != 'zip':
        final_food_df[column] = final_food_df[column].apply(
    lambda x: (x-min(final_food_df[column]))/(max(final_food_df[column])-min(final_food_df[column])))
        
final_food_df.reset_index(inplace = True)

In [93]:
#Display dataframe
final_food_df.head()

category,zip,Carry Out,Farmers Market,Market,Restaurant
0,20705,0.0,0.0,0.008197,0.018868
1,20783,0.0,0.0,0.000000,0.006289
2,20810,0.0,0.0,0.000000,0.003774
3,20811,0.0,0.0,0.000000,0.006289
4,20812,0.0,0.0,0.000000,0.005031


## SDAT Dataset

In [53]:
#create a list of desired fields
fields = ['premise_address_number_mdp_field_premsnum_sdat_field_20','premise_address_name_mdp_field_premsnam_sdat_field_23',
          'premise_address_type_mdp_field_premstyp_sdat_field_24','premise_address_city_mdp_field_premcity_sdat_field_25',
          'premise_address_zip_code_mdp_field_premzip_sdat_field_26',
          'sales_segment_1_transfer_date_yyyy_mm_dd_mdp_field_tradate_sdat_field_89',
          'sales_segment_1_consideration_mdp_field_considr1_sdat_field_90',
          'land_use_code_mdp_field_lu_desclu_sdat_field_50']

#convert list of desired fields into a string to be set as an arguement in API Call
select_statement = ','.join(fields)

#authenticate client 
client = Socrata("opendata.maryland.gov",credentials.data_moco_token,
                 credentials.data_moco_username,credentials.data_moco_password)

#API Call for all properties in Montgomery County
sdat_results = client.get_all("ed4q-f8tm", where='county_name_mdp_field_cntyname = "Montgomery County"',
                               select = select_statement)
                            

# Convert to pandas DataFrame
sdat_df = pd.DataFrame.from_records(sdat_results)

#Rename columns
sdat_df= sdat_df.rename(columns={"premise_address_number_mdp_field_premsnum_sdat_field_20":"stno",
                     'premise_address_name_mdp_field_premsnam_sdat_field_23':"stname",
                     'premise_address_type_mdp_field_premstyp_sdat_field_24':"suffix",
                     'premise_address_city_mdp_field_premcity_sdat_field_25': 'city', 
                     'premise_address_zip_code_mdp_field_premzip_sdat_field_26': 'zip',
                     'sales_segment_1_transfer_date_yyyy_mm_dd_mdp_field_tradate_sdat_field_89': 'sales_date',
                     'sales_segment_1_consideration_mdp_field_considr1_sdat_field_90': 'sales_price',
                     'land_use_code_mdp_field_lu_desclu_sdat_field_50': 'land_use_code'})
#Display dataframe
sdat_df.head()

,stno,stname,suffix,zip,sales_date,sales_price,land_use_code,city
0,00000,TWIN VALLEY,CT,00000,0000.00.00,0,Exempt (E),NaN
1,00000,CEDAR,LN,20895,0000.00.00,0,Exempt (E),KENSINGTON
2,00000,DAMASCUS,RD,20882,2005.10.05,0,Residential (R),GAITHERSBURG
3,00000,HOWARD CHAPEL,RD,00000,0000.00.00,0,Commercial (C),NaN
4,00000,GOSHEN,RD,00000,0000.00.00,0,Residential (R),NaN


## Clean SDAT Data

In [98]:
#drop duplicates
sdat_df_no_dups = sdat_df.drop_duplicates(inplace=False)

#change date from '0000.00.00' to '1800.01.01' for processing purposes 
sdat_df_no_dups.replace('0000.00.00','1800.01.01', inplace = True)

#create dict to fill null values in sales price column
values = {"sales_price": 0}

#fill null values in sales price column
sdat_df_adjust_2 = sdat_df_no_dups.fillna(value=values, inplace=False)

#convert sales date from a string to datetime format
sdat_df_adjust_2['sales_date'] = pd.to_datetime(sdat_df_adjust_2['sales_date'])

#convert street number from string to interger to remove zeros
sdat_df_adjust_2['sales_price'] = sdat_df_adjust_2['sales_price'].astype('float')

#filter for properties sold after 1/1/2016 and is zip codes match the moco_zip list
filtered_sdat_df = sdat_df_adjust_2.loc[(sdat_df_adjust_2['sales_date'] >= datetime.datetime(2016, 1, 1)) &
                                       (sdat_df_adjust_2.zip.isin(moco_zips))]

#filter for only residential properties
filtered_sdat_df_2 = filtered_sdat_df.loc[filtered_sdat_df.sales_price > 0]

#filter for only residential properties
residential_sdat_df = filtered_sdat_df_2.loc[filtered_sdat_df_2.land_use_code =='Residential (R)']

#create table to show median home prices by zip code
final_res_median_sale_price_df = residential_sdat_clean_df.groupby('zip')['sales_price'].agg('median').to_frame().reset_index()

#normalize median home prices
for column in final_res_median_sale_price_df.columns:
    if column != 'zip':
        final_res_median_sale_price_df[column] = final_res_median_sale_price_df[column].apply(
    lambda x: (x-min(final_res_median_sale_price_df[column]))/(max(final_res_median_sale_price_df[column])
                                                               -min(final_res_median_sale_price_df[column])))

C:\Users\mivul\anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [95]:
#display dataframe
final_res_median_sale_price_df.head()

,zip,sales_price
0,20705,0.000000
1,20707,0.338078
2,20777,0.306588
3,20783,0.135231
4,20812,0.629893


## Crime Dataset

In [63]:
#authenticate client
client = Socrata('data.montgomerycountymd.gov',credentials.data_moco_token,
                 credentials.data_moco_username,credentials.data_moco_password)

#API Call for all reported crimes
reported_crime = client.get_all("icn6-v9z3")

#convert response to dataframe
crime_df = pd.DataFrame.from_records(reported_crime)

#display dataframe
crime_df.head()

,incident_id,offence_code,case_number,date,nibrs_code,victims,crimename1,crimename2,crimename3,district,location,city,state,zip_code,agency,place,sector,beat,pra,address_number,address_street,street_type,start_date,end_date,latitude,longitude,police_district_number,geolocation,:@computed_region_vu5j_pcmz,:@computed_region_tx5f_5em3,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n,:@computed_region_a9cs_3ed7,:@computed_region_r648_kzwt,:@computed_region_d9ke_fpxt,street_prefix_dir,street_suffix_dir
0,201378490,2610,220023328,2022-05-31T19:16:36.000,26F,1,Crime Against Property,Identity Theft,IDENTITY THEFT,SILVER SPRING,700 BLK SLIGO AVE,SILVER SPRING,MD,20910,MCPD,Other/Unknown,G,3G1,515,700,SLIGO,AVE,2020-10-02T00:00:00.000,2020-10-31T00:00:00.000,38.9904,-77.0169,3D,"{'latitude': '38.9904', 'longitude': '-77.0169...",5,5,7,3065,1,5,1,1,NaN,NaN
1,201378504,3605,220023331,2022-05-31T19:14:53.000,90C,1,Crime Against Society,All Other Offenses,SEX OFFENSE - INDECENT EXPOSURE,ROCKVILLE,600 BLK EDMONSTON DR,ROCKVILLE,MD,20851,RCPD,Street - Commercial,A,1A2,288,600,EDMONSTON,DR,2022-05-31T18:14:00.000,2022-05-31T18:20:00.000,39.08394,-77.1303,1D,"{'latitude': '39.0839', 'longitude': '-77.1303...",3,3,25,68,12,3,3,1,NaN,NaN
2,201378486,9105,220023325,2022-05-31T19:02:09.000,90Z,1,Other,All Other Offenses,LOST PROPERTY,MONTGOMERY VILLAGE,NaN,GAITHERSBURG,MD,20886,MCPD,Street - Residential,R,6R2,504,NaN,FERN HOLLOW,WAY,2021-11-20T00:00:00.000,2021-11-20T23:59:00.000,0.0,0.0,6D,"{'latitude': '0.0', 'longitude': '0.0', 'human...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201378481,9199,220023313,2022-05-31T17:33:22.000,90Z,1,Other,All Other Offenses,POLICE INFORMATION,MONTGOMERY VILLAGE,19300 BLK WATKINS MILL RD,MONTGOMERY VILLAGE,MD,20886,MCPD,School - Elementary/Secondary,R,6R2,424,19300,WATKINS MILL,RD,2022-05-31T11:00:00.000,2022-05-31T11:15:00.000,39.17265,-77.2091,6D,"{'latitude': '39.1726', 'longitude': '-77.2091...",2,2,38,28,1,2,5,1,NaN,NaN
4,201378477,2601,220023310,2022-05-31T17:31:21.000,26A,1,Crime Against Property,False Pretenses/Swindle/Confidence Game,FRAUD - CONFIDENCE GAME,ROCKVILLE,17200 BLK CHISWELL RD,POOLESVILLE,MD,20837,MCPD,Residence - Single Family,B,1B3,499,17200,CHISWELL,RD,2022-05-31T10:00:00.000,2022-05-31T15:10:00.000,39.13798,-77.4111,1D,"{'latitude': '39.138', 'longitude': '-77.4111'...",1,1,34,97,16,1,5,1,NaN,NaN


## Clean Crime Dataset

In [99]:
#change column type from string to datetime
crime_df['date'] = pd.to_datetime(crime_df['date'])

#drop duplicate cases
crime_df_2 = crime_df.drop(['geolocation'],axis='columns', inplace = False)
crime_df_no_dups = crime_df_2.drop_duplicates(inplace=False)

#filter dataset to keep entries with zip codes in moco_zips
crime_3 = crime_df_no_dups.loc[crime_df_no_dups.zip_code.isin(moco_zips)]

#filter for properties sold after 1/1/2016
filtered_crime_df = crime_3.loc[crime_3['date'] >= datetime.datetime(2016, 1, 1)]

#type of crimes by zipcode
final_crime_df=filtered_crime_df.groupby('zip_code')["crimename2"].value_counts().unstack().fillna(0).reset_index()

#normalize median home prices
for column in final_crime_df.columns:
    if column != 'zip_code':
        final_crime_df[column] = final_crime_df[column].apply(
    lambda x: (x-min(final_crime_df[column]))/(max(final_crime_df[column])-min(final_crime_df[column])))

In [68]:
#display dataframe
final_crime_df.head()

crimename2,zip_code,Aggravated Assault,All Other Offenses,All other Larceny,Arson,Assisting or Promoting Prostitution,Bad Checks,Bribery,Burglary/Breaking and Entering,Counterfeiting/Forgery,Credit Card/Automatic Teller Machine Fraud,Curfew/Loitering/Vagrancy Violations,Destruction/Damage/Vandalism of Property,Disorderly Conduct,Driving Under the Influence,Drug Equipment Violations,Drug/Narcotic Violations,Drunkenness,Embezzlement,Extortion/Blackmail,False Pretenses/Swindle/Confidence Game,"Family Offenses, NonViolent",Fondling,Forcible Fondling,Forcible Rape,Forcible Sodomy,From Coin/Operated Machine or Device,"Human Trafficking, Commercial Sex Acts","Human Trafficking, Involuntary Servitude",Identity Theft,Impersonation,Incest,Intimidation,Justifiable Homicide,Kidnapping/Abduction,Liquor Law Violations,Motor Vehicle Theft,Murder and Nonnegligent Manslaughter,NOT NIBRS CODE,Negligent Manslaughter,Not Mapped,Operating/Promoting/Assisting Gambling,Peeping Tom,Pocket/picking,Pornography/Obscene Material,Prostitution,Purse-snatching,Robbery,Runaway,Sexual Assault With An Object,Shoplifting,Simple Assault,Statuory Rape,Stolen Property Offenses,Theft From Motor Vehicle,Theft from Building,Theft of Motor Vehicle Parts or Accessories,Trespass of Real Property,Weapon Law Violations,Welfare Fraud,Wire Fraud
0,20705,0.013812,0.004580,0.000000,0.0,0.0,0.0,0.0,0.007533,0.004484,0.000000,0.0,0.001442,0.0,0.001750,0.0,0.001004,0.0,0.000000,0.0,0.002667,0.0,0.000000,0.0,0.011905,0.02381,0.0,0.0,0.0,0.004918,0.000000,0.0,0.0,0.0,0.0,0.0,0.005722,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.003344,0.027778,0.001026,0.005003,0.0,0.0,0.000932,0.0000,0.000000,0.001754,0.005848,0.0,0.0
1,20707,0.002762,0.002835,0.002043,0.0,0.0,0.0,0.0,0.013183,0.000000,0.000000,0.0,0.003605,0.0,0.001750,0.0,0.000502,0.0,0.030303,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.004918,0.000000,0.0,0.0,0.0,0.0,0.0,0.001431,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.002331,0.0011,0.003571,0.000000,0.000000,0.0,0.0
2,20777,0.000000,0.000872,0.001021,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000721,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000556,0.0,0.0,0.000466,0.0000,0.000000,0.000000,0.000000,0.0,0.0
3,20783,0.000000,0.002617,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000721,0.0,0.005249,0.0,0.004518,0.0,0.000000,0.0,0.000000,0.0,0.014286,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.001431,0.0,0.038462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002681,0.000000,0.000000,0.000513,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
4,20812,0.000000,0.001963,0.008172,0.0,0.0,0.0,0.0,0.000000,0.026906,0.019531,0.0,0.000721,0.0,0.000875,0.0,0.000000,0.0,0.000000,0.0,0.005333,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.004918,0.018868,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.001112,0.0,0.0,0.006061,0.0000,0.001786,0.000000,0.000000,0.0,0.0


## Great Schools Dataset

In [100]:
#initialize a blank dataframe to fill with school information
blank_df = pd.DataFrame(columns=['school_name','school_rating','school_zip'])

#initialize seconds to wait 2 seconds between API calls
seconds = 2

In [101]:
#for loop to iterate through zip code list to input into great schools API call 
for zip_code in moco_zips:
    try:
        #Call API and read the school data by zip code
        url = f'https://gs-api.greatschools.org/schools?zip={zip_code}&limit=50'
        req = Request(url)
        req.add_header('X-API-Key', credentials.great_school_key)
        content = urlopen(req).read()
        school_df = pd.read_json(content)

        #Acquire the school name, zip code, and school rating from  
        school_names = [row.schools.get('name') for i , row in school_df.iterrows()]
        school_zips = [row.schools.get('zip') for i , row in school_df.iterrows()]
        school_rating = [row.schools.get('rating') for i , row in school_df.iterrows()] 

        #Create a dataframe from the acquired data and concatinate it with the initialized blank dataframe
        clean_school_df= pd.DataFrame(zip(school_names,school_rating,school_zips ), 
                                  columns=['school_name','school_rating','school_zip'])
        #fill in a 0 for schools with no ratings
        clean_school_df.fillna(0, inplace = True)
        blank_df = pd.concat([blank_df,clean_school_df], ignore_index=True)
    
    except:
        print(f'{zip_code} had an error, and will not be added to dataframe.')
    
    #wait 2 seconds between calls
    sleep(seconds)

#display dataframe
blank_df.head()

,school_name,school_rating,school_zip
0,Beltsville Academy,5,20705
1,Calverton Elementary School,4,20705
2,High Point High School,3,20705
3,Martin Luther King Jr. Middle School,5,20705
4,Beltsville Sda School,0,20705


## Clean Great Schools Dataset

In [102]:
#copy the final dataframe with all of the school information
all_schools_df = blank_df.copy()

#convert values from str to int
all_schools_df['school_rating'] = all_schools_df['school_rating'].astype('int')

#replace zero values with the avg school rating value for all Montgomery county
all_schools_df['school_rating'].replace(0, all_schools_df['school_rating'].mean(), inplace= True)

#save dataframe as csv
#all_schools_df.to_csv("great_schools.csv", index=False)

In [103]:
#calculate average rating for all schools by zip
all_schools_rating_df= all_schools_df.groupby('school_zip')['school_rating'].agg('mean').to_frame().reset_index()

#normalize average school rating data 
for column in all_schools_rating_df.columns:
    if column != 'school_zip':
        all_schools_rating_df[column] = all_schools_rating_df[column].apply(
    lambda x: (x-min(all_schools_rating_df[column]))/(max(all_schools_rating_df[column])-min(all_schools_rating_df[column])))

#display dataframe
all_schools_rating_df.head()

,school_zip,school_rating
0,20705,0.142635
1,20707,0.140715
2,20783,0.170072
3,20814,0.224672
4,20815,0.454855


## Census Dataset

In [76]:
#initiate an empty list to collect api information
median_income_list = []

#iterare through the zip code list to input in Census API to get the median household income
for zip_code in moco_zips:
    try:
        url =f'https://api.census.gov/data/2020/acs/acs5?get=NAME,B19013_001E&for=zip%20code%20tabulation%20area:{zip_code}&key={credentials.census_key}'
        #API Call 
        response = requests.request("GET", url)
        #convert json to list
        result = json.loads(response.text)
        #add results to initial empty list
        median_income_list.append(result[1])
    except:
        print(f'{zip_code} had an error, and will not be added to dataframe.')

#convert median_income_list to a dataframe
median_income_df= pd.DataFrame(median_income_list, columns =['zcta', 'median_income', 'zip'])

20813 had an error
20824 had an error
20825 had an error
20827 had an error
20830 had an error
20847 had an error
20848 had an error
20849 had an error
20859 had an error
20875 had an error
20883 had an error
20884 had an error
20885 had an error
20891 had an error
20898 had an error
20907 had an error
20908 had an error
20911 had an error
20913 had an error
20914 had an error
20915 had an error
20916 had an error
20918 had an error
20810 had an error
20811 had an error
20857 had an error
20889 had an error
20892 had an error
20894 had an error
20897 had an error
20993 had an error
20997 had an error


In [77]:
#display dataframe
median_income_df.head()

,zcta,median_income,zip
0,ZCTA5 20705,92257,20705
1,ZCTA5 20707,85935,20707
2,ZCTA5 20777,198111,20777
3,ZCTA5 20783,66905,20783
4,ZCTA5 20812,141250,20812


## Clean Census Dataset

In [78]:
#convert values in median_income column from str to int
median_income_df['median_income'] =  median_income_df['median_income'].astype('int')

#change median income value for zip code 20868 to 0 because median value is negative
median_income_df.at[27,'median_income']=0

#drop the zcta column
final_median_income_df = median_income_df.drop(['zcta'], axis = 1)

#normalize median income 
for column in final_median_income_df.columns:
    if column != 'zip':
        final_median_income_df[column] = final_median_income_df[column].apply(
    lambda x: (x-min(final_median_income_df[column]))/(max(final_median_income_df[column])-min(final_median_income_df[column])))

In [79]:
#display dataframe
final_median_income_df.head()

,median_income,zip
0,0.437986,20705
1,0.407973,20707
2,0.940524,20777
3,0.317629,20783
4,0.670579,20812


# Master table

In [104]:
def weighted_avg(df):
    """ Calculate the weighted average of the normalized dataset with a weight of 1/6. Each dataset value will be 
    weighted equally."""
    df_copy = df.copy()
    for column in df.columns:
        if column in final_median_income_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(final_median_income_df.columns)-1))
        elif column in final_res_median_sale_price_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(final_res_median_sale_price_df.columns)-1))
        elif column in all_schools_rating_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(all_schools_rating_df.columns)-1))
        elif column in final_crime_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(final_crime_df.columns)-1))
        elif column in final_food_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(final_food_df.columns)-1))
        elif column in final_construction_df.columns:
            df_copy[column] = df_copy[column].apply(lambda x: x * (1/6)/(len(final_construction_df.columns)-1))
    df_copy['grand_total'] = df_copy.agg('sum', axis = 1)
    return df_copy['grand_total']  

In [107]:
#merge all dataframes together based on zipcode
step_1 = final_median_income_df.merge(final_res_median_sale_price_df,how='left', on ='zip')
step_2 = step_1.merge(all_schools_rating_df,how='left', left_on='zip', right_on='school_zip')
step_3 = step_2.merge(final_crime_df,how='left', left_on='zip', right_on='zip_code')
step_4 = step_3.merge(final_food_df,how='left', on ='zip')
step_5 = step_4.merge(final_construction_df,how='left', on ='zip')

#drop zip_code and school_zip columns
step_5.drop(['zip_code','school_zip'],axis=1, inplace = True)

#fill na with zeros and set the zip column as an index
master_table = step_5.fillna(0).set_index('zip')

#sum accross the rows to calculate grand_total value
master_table['index_value'] = weighted_avg(master_table)

#sort values by grand_total column in descending order
final_master_table = master_table.sort_values('index_value', ascending=False)

#reset the zip index as a column
final_master_table.reset_index(inplace = True)

In [110]:
#save dataframe as a CSV
final_master_table.to_csv("master.csv", index=False)

#display dataframe
final_master_table.head()

,zip,median_income,sales_price,school_rating,Aggravated Assault,All Other Offenses,All other Larceny,Arson,Assisting or Promoting Prostitution,Bad Checks,Bribery,Burglary/Breaking and Entering,Counterfeiting/Forgery,Credit Card/Automatic Teller Machine Fraud,Curfew/Loitering/Vagrancy Violations,Destruction/Damage/Vandalism of Property,Disorderly Conduct,Driving Under the Influence,Drug Equipment Violations,Drug/Narcotic Violations,Drunkenness,Embezzlement,Extortion/Blackmail,False Pretenses/Swindle/Confidence Game,"Family Offenses, NonViolent",Fondling,Forcible Fondling,Forcible Rape,Forcible Sodomy,From Coin/Operated Machine or Device,"Human Trafficking, Commercial Sex Acts","Human Trafficking, Involuntary Servitude",Identity Theft,Impersonation,Incest,Intimidation,Justifiable Homicide,Kidnapping/Abduction,Liquor Law Violations,Motor Vehicle Theft,Murder and Nonnegligent Manslaughter,NOT NIBRS CODE,Negligent Manslaughter,Not Mapped,Operating/Promoting/Assisting Gambling,Peeping Tom,Pocket/picking,Pornography/Obscene Material,Prostitution,Purse-snatching,Robbery,Runaway,Sexual Assault With An Object,Shoplifting,Simple Assault,Statuory Rape,Stolen Property Offenses,Theft From Motor Vehicle,Theft from Building,Theft of Motor Vehicle Parts or Accessories,Trespass of Real Property,Weapon Law Violations,Welfare Fraud,Wire Fraud,Carry Out,Farmers Market,Market,Restaurant,total,index_value
0,20814,0.654969,0.697509,0.224672,0.138122,0.419847,0.569969,0.173913,0.0,0.318182,0.0,0.397363,1.000000,0.796875,0.0,0.345350,0.223329,0.300087,0.108911,0.170181,0.166667,0.636364,0.37500,0.504000,0.125000,0.400000,0.100,0.321429,0.333333,0.0,0.055556,0.0,0.572131,0.792453,0.0,0.477273,0.000000,0.2,0.106265,0.238913,0.142857,0.192308,1.0,0.964286,0.0,0.277778,0.626667,0.019048,0.078947,0.250000,0.109920,0.073579,0.194444,0.164700,0.251807,0.00,0.000000,0.351515,0.602860,0.360714,0.101754,0.152047,0.0,0.333333,1.000000,1.0,0.590164,0.847799,0.214909,0.488078
1,20871,0.647620,0.247785,0.702360,0.080110,0.173610,0.271706,0.434783,0.0,0.181818,0.0,0.165725,0.094170,0.277344,0.0,0.144917,0.021337,0.069991,0.059406,0.069277,0.000000,0.242424,0.25000,0.216000,0.152778,0.171429,0.125,0.202381,0.214286,0.0,0.000000,0.0,0.313115,0.150943,0.0,0.159091,0.000000,0.0,0.009516,0.042918,0.214286,0.038462,0.0,0.000000,0.0,0.166667,0.040000,0.000000,0.000000,0.068182,0.061662,0.123746,0.111111,0.212417,0.154530,0.25,0.086957,0.102564,0.167217,0.080357,0.035088,0.192982,0.0,0.095238,0.113636,0.0,0.200820,0.122013,1.000000,0.469190
2,20817,0.980787,0.572954,0.443913,0.088398,0.381243,0.319714,0.130435,0.0,0.409091,0.0,0.429379,0.690583,0.589844,0.0,0.240087,0.031294,0.082240,0.094059,0.125502,0.111111,0.424242,0.53125,0.512000,0.180556,0.271429,0.225,0.214286,0.214286,0.0,0.000000,0.0,0.608197,0.962264,0.0,0.272727,0.000000,0.2,0.021412,0.187411,0.000000,0.115385,0.0,0.142857,0.0,0.222222,0.466667,0.038095,0.000000,0.159091,0.085791,0.578595,0.138889,0.473063,0.169539,0.00,0.173913,0.354779,0.437844,0.248214,0.054386,0.093567,0.0,0.333333,0.204545,0.0,0.389344,0.313208,0.358049,0.466064
3,20815,0.773912,0.839858,0.454855,0.063536,0.223991,0.356486,0.260870,0.0,0.272727,0.0,0.344633,0.645740,0.515625,0.0,0.181687,0.031294,0.093613,0.039604,0.067771,0.000000,0.424242,0.43750,0.373333,0.083333,0.142857,0.100,0.154762,0.166667,0.0,0.000000,0.0,0.440984,0.849057,0.0,0.250000,0.000000,0.2,0.020619,0.206009,0.071429,0.115385,0.0,0.107143,0.0,0.055556,0.573333,0.019048,0.000000,0.250000,0.072386,0.073579,0.083333,0.180092,0.091718,0.00,0.130435,0.378089,0.361936,0.255357,0.029825,0.017544,0.0,0.238095,0.113636,0.0,0.229508,0.129560,0.151071,0.417565
4,20854,1.000000,0.537367,0.321583,0.093923,0.327808,0.286006,0.173913,0.0,0.409091,0.0,0.369115,0.403587,0.628906,0.0,0.225667,0.031294,0.125984,0.054455,0.086847,0.000000,0.303030,0.53125,0.613333,0.097222,0.171429,0.100,0.130952,0.142857,0.0,0.055556,0.0,0.854098,1.000000,0.0,0.272727,0.333333,0.2,0.020619,0.143062,0.071429,0

## Sources

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://www.data.montgomerycountymd.gov 
- https://opendata.maryland.gov/
- https://www.greatschools.org/
- https://maryland.hometownlocator.com/zip-codes/countyzips,scfips,24031,c,montgomery.cfm